In [1]:
def knapsack_backtracking(weights, values, capacity):
    n = len(weights)

    def backtrack(index, current_weight, current_value):
        if index == n or current_weight > capacity:
            return current_value if current_weight <= capacity else 0
        
        # Pilih atau tidak pilih barang ke-i
        include = backtrack(index + 1, current_weight + weights[index], current_value + values[index])
        exclude = backtrack(index + 1, current_weight, current_value)
        return max(include, exclude)

    return backtrack(0, 0, 0)

# Contoh penggunaan
weights = [2, 3, 4, 10]
values = [3, 4, 5, 8]
capacity = 6
print(knapsack_backtracking(weights, values, capacity))


8


Penjelasan:

- Ide Utama: Eksplorasi semua kombinasi barang yang mungkin untuk dimasukkan ke dalam ransel.
- Proses Kerja:
  1. Rekursif memeriksa setiap barang, dengan dua pilihan:
  - Pilih barang: Tambahkan berat dan nilai barang tersebut.
  - Tidak pilih barang: Lanjutkan ke barang berikutnya tanpa menambah berat atau nilai.
  2. Ketika suatu kombinasi barang melampaui kapasitas, algoritma berhenti menjelajahi cabang tersebut (pruning).
  3. Nilai maksimum dicatat sepanjang proses.
- Kelebihan: Dapat menemukan solusi optimal.
- Kekurangan: Lambat untuk input besar karena kompleksitas eksponensial.

In [2]:
import heapq

class Node:
    def __init__(self, level, value, weight, bound):
        self.level = level
        self.value = value
        self.weight = weight
        self.bound = bound

    def __lt__(self, other):
        return self.bound > other.bound  # Max heap (by bound)

def knapsack_branch_and_bound(weights, values, capacity):
    def bound(node):
        if node.weight >= capacity:
            return 0
        profit_bound = node.value
        total_weight = node.weight
        for i in range(node.level + 1, len(weights)):
            if total_weight + weights[i] <= capacity:
                total_weight += weights[i]
                profit_bound += values[i]
            else:
                profit_bound += (capacity - total_weight) * (values[i] / weights[i])
                break
        return profit_bound

    n = len(weights)
    queue = []
    root = Node(-1, 0, 0, 0)
    root.bound = bound(root)
    heapq.heappush(queue, root)
    max_value = 0

    while queue:
        current = heapq.heappop(queue)
        if current.bound > max_value:
            next_level = current.level + 1

            # Pilih barang
            if next_level < n:
                left = Node(next_level, current.value + values[next_level],
                            current.weight + weights[next_level], 0)
                if left.weight <= capacity:
                    max_value = max(max_value, left.value)
                left.bound = bound(left)
                if left.bound > max_value:
                    heapq.heappush(queue, left)

            # Tidak pilih barang
            right = Node(next_level, current.value, current.weight, 0)
            right.bound = bound(right)
            if right.bound > max_value:
                heapq.heappush(queue, right)

    return max_value

# Contoh penggunaan
weights = [2, 3, 4, 10]
values = [3, 4, 5, 8]
capacity = 6
print(knapsack_branch_and_bound(weights, values, capacity))


8


Penjelasan:

- Ide Utama: Menggunakan bound (batas maksimum keuntungan yang bisa diperoleh dari simpul tertentu) untuk memangkas cabang yang tidak mungkin menghasilkan solusi optimal.
- Proses Kerja:
- Bounding: Setiap simpul pohon dihitung bound-nya, yaitu estimasi nilai maksimum yang bisa diperoleh dari simpul tersebut.
- Jika bound suatu simpul lebih kecil dari nilai maksimum yang sudah ditemukan, maka cabang tersebut tidak dieksplorasi lebih lanjut (pruning).
- Algoritma menggunakan priority queue untuk memprioritaskan simpul dengan bound tertinggi.
- Kelebihan: Lebih cepat dibanding backtracking karena memanfaatkan teknik pemangkasan.
- Kekurangan: Kompleksitas masih tinggi untuk kasus besar.

In [3]:
def knapsack_greedy(weights, values, capacity):
    n = len(weights)
    items = sorted([(values[i] / weights[i], weights[i], values[i]) for i in range(n)], reverse=True)

    total_value = 0
    for ratio, weight, value in items:
        if capacity >= weight:
            capacity -= weight
            total_value += value
        else:
            total_value += ratio * capacity
            break

    return total_value

# Contoh penggunaan
weights = [2, 3, 4, 10]
values = [3, 4, 5, 8]
capacity = 6
print(knapsack_greedy(weights, values, capacity))


8.25


Penjelasan:

- Ide Utama: Pilih barang berdasarkan rasio nilai per berat tertinggi (value-to-weight ratio).
- Proses Kerja:
- Barang diurutkan berdasarkan rasio nilai per berat.
- Mulai dari barang dengan rasio tertinggi, tambahkan ke ransel hingga kapasitas penuh.
- Jika kapasitas tersisa tidak mencukupi untuk barang berikutnya, tambahkan sebagian barang (jika diperbolehkan).
- Kelebihan: Sangat cepat, kompleksitas rendah (O(n log n) karena pengurutan).
- Kekurangan: Tidak selalu menghasilkan solusi optimal untuk semua kasus (hanya optimal untuk masalah tertentu, seperti fractional knapsack).